<a href="https://colab.research.google.com/github/SalFell/CPSC-483-Machine-Learning/blob/main/Discussion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive,files
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import sklearn.linear_model
drive.mount('/content/drive/')
#uploaded = files.upload()

hls_all_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/HSL_08022024200038350.csv")

#print(hls_all_raw["Indicator"])
print("\n1===========================================================\n")
hls_slice = pd.DataFrame(hls_all_raw, columns =["Country","Indicator","Type of indicator","Time","Value"])
#print(hls_slice)
#
hls_ls = hls_slice.loc[hls_all_raw["Indicator"] == "Employment rate"]
print(hls_ls)
print("\n2===========================================================\n")
print("Total records:")
print(len(hls_ls))

print("\n3===========================================================\n")
print("Total Unique Countries:")
print(len(hls_ls["Country"].unique()))

print("\n4===========================================================\n")
print("Country List")
print(hls_ls["Country"].unique())
#
hls_train = hls_ls.loc[hls_ls["Time"] == 2021]
#hls_train = hls_ls.loc[hls_ls["Time"] >= 2018]
hls_train = hls_train.loc[hls_ls["Type of indicator"] == "Average"]
print("\n5===========================================================\n")
print("Total records:")
print(len(hls_train))

print("\n6===========================================================\n")
print("Total Unique Countries:")
print(len(hls_train["Country"].unique()))

print("\n7===========================================================\n")
print("Record:")
print(hls_train)

#
weo_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/WEOOct2023all.xls", sep='\t', encoding='latin1')
#
weo_selected_measurement = weo_raw.loc[weo_raw['WEO Subject Code'].str.contains("NGDP_RPCH", na=False)]
weo_selected_measurement_2018 = pd.DataFrame(weo_selected_measurement, columns=['Country', '2018'])

print(weo_selected_measurement_2018)

#
merged_train_data = pd.merge(hls_train, weo_selected_measurement_2018, on="Country")
merged_train_data = merged_train_data.rename(columns={"Value": "Happiness Measurement", "2018": "Employment Measurement"})
merged_train_data = pd.DataFrame(merged_train_data, columns=['Country','Happiness Measurement', 'Employment Measurement'])

print(merged_train_data)

#
X = np.c_[merged_train_data["Employment Measurement"]]
Y = np.c_[merged_train_data["Happiness Measurement"]]
x = X.tolist()
y = Y.tolist()

# plot data
out1 = widgets.Output()
with out1:
  plt.scatter(x, y)
  plt.xlabel('Employment')
  plt.ylabel('Happiness')
  plt.title("Data Plot")
  plt.show()

# fit linear model
model = sklearn.linear_model.LinearRegression()
model.fit(X, Y)

# plot predictions
predict_x = [x for x in range(901)]
predict_x = [[x/100] for x in predict_x]
predict_y = model.predict(predict_x)

out2 = widgets.Output()
with out2:
  plt.scatter(predict_x, predict_y)
  plt.scatter(x, y)
  plt.xlabel('Employment')
  plt.ylabel('Happiness')
  plt.title("Prediction Line")
  plt.show()

display(widgets.HBox([out1,out2]))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).

1===========================================================

            Country        Indicator Type of indicator  Time      Value
21        Australia  Employment rate           Average  2004  72.397004
22        Australia  Employment rate           Average  2005  73.679393
23        Australia  Employment rate           Average  2006  74.308828
24        Australia  Employment rate           Average  2007  75.057021
25        Australia  Employment rate           Average  2008  75.475192
...             ...              ...               ...   ...        ...
12048  South Africa  Employment rate           Average  2017  55.148640
12049  South Africa  Employment rate           Average  2018  54.971014
12050  South Africa  Employment rate           Average  2019  53.920095
12051  South Africa  Employment rate           Average  2020  49.210919
12052  South A

In [6]:
# Discussion 2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display

# Assuming merged_train_data is your dataset

X = np.array(merged_train_data["Employment Measurement"]).reshape(-1, 1)
Y = np.array(merged_train_data["Happiness Measurement"]).reshape(-1, 1)
x = X.flatten()
y = Y.flatten()

# plot data
out1 = widgets.Output()
with out1:
    plt.scatter(x, y)
    plt.xlabel('Employment')
    plt.ylabel('Happiness')
    plt.title("Data Plot")
    plt.show()

# Linear Regression
model_linear = LinearRegression()
model_linear.fit(X, Y)
predict_y_linear = model_linear.predict(X)

# Ridge Regression
model_ridge = Ridge(alpha=1, solver="cholesky")
model_ridge.fit(X, Y)
predict_y_ridge = model_ridge.predict(X)

# Another model (e.g., Decision Tree)
# Replace the model and hyperparameters as needed
model_other = DecisionTreeRegressor()
model_other.fit(X, Y)
predict_y_other = model_other.predict(X)

# Calculate mean squared errors
mse_linear = mean_squared_error(Y, predict_y_linear)
mse_ridge = mean_squared_error(Y, predict_y_ridge)
mse_other = mean_squared_error(Y, predict_y_other)

# plot predictions
out2 = widgets.Output()
with out2:
    plt.scatter(x, y, label='Actual Data')
    plt.scatter(x, predict_y_linear, label=f'Linear Regression (MSE: {mse_linear:.2f})')
    plt.scatter(x, predict_y_ridge, label=f'Ridge Regression (MSE: {mse_ridge:.2f})')
    plt.scatter(x, predict_y_other, label=f'Decision Tree (MSE: {mse_other:.2f})')

    plt.xlabel('Employment')
    plt.ylabel('Happiness')
    plt.title("Prediction Lines")
    plt.legend()
    plt.show()

display(widgets.HBox([out1, out2]))